# Ansible

 - configuration manager 
 - simple 
 - extensible via modules
 - written in python
 - broad community
 - many external tools
 - playbook repository
 - used by openstack, openshift & tonns of project
 
 
 # Configuration Manager
 
 bla bla bla infrastructure as code
 
 # Advantages
 
 No agents: ansible copies python and all deployment scripts/modules to the target machine via ssh and executes them remotely. Some modules though require that target hosts contain specific python libraries.
 
 Jobs are executed in parallel, but you can configure for serialization using different strategies for speed up, rollout or other purposes: (link)
 
 Authentication can be passwordless (ssh/pki, kerberos) or with password.
 
 Automation jobs (Playbooks) are described via YAML - a very concise and simple language. You can validate and lint files with yamllint and ansible-lint
 
 Passwords are supported, but SSH keys with ssh-agent are one of the best ways to use Ansible. Though if you want to use Kerberos, that's good too. Lots of options! Root logins are not required, you can login as any user, and then su or sudo to any user.




In [2]:
cd ansible

[Errno 2] No such file or directory: 'ansible'
/notebooks/notebooks/ansible


In [12]:
# Let's check our ansible directory
!ls -l

total 28
-rw-rw-r--. 1 1000 1000 904 Jun  9 21:57 ansible.cfg
-rw-rw-r--. 1 1000 1000 514 Jul 18  2016 inventory
-rw-rw-r--. 1 1000 1000 188 Jul 18  2016 mytemplate.j2
-rw-r--r--. 1 1000 1000  27 Jun  9 21:27 python-course-test.retry
-rw-rw-r--. 1 1000 1000 505 Jul 18  2016 python-course-test.yml
-rw-r--r--. 1 1000 1000  27 Jun  9 21:27 site.retry
-rw-rw-r--. 1 1000 1000 448 Jul 18  2016 site.yml


## ansible.cfg 

In [13]:
# It's the main configuration file: we'll check it in the following lesson
!cat ansible.cfg


#
# Always use ansible.cfg and inventory files to 
#  describe your deployment! 
#
# This will save time and helps your colleagues to
#  keep in touch with the environment.
#
[defaults]
# define our inventory file or ansible defaults
#  to /etc/ansible/hosts.
inventory = inventory

# It's just for testing purposes (or if you're on a secure cloud)
#  you may want to avoid typing yes to all host keys.
# Once you get host keys, you can comment this out and re-enable
#  checks.
host_key_checking = False

# When an ansible plabook fails, it creates a retry file which may
#  pollute the current directory ;) You can move this out using
#  further variables
retry_files_enabled = False

# Search password file in here
# vault_password_file = ~/.ssh/pin


# [ssh_connection]
# ssh_args = -F ./ssh_config/config -o ControlMaster=auto -o ControlPersist=30m
# control_path = ./ssh_config/ansible-%%r@%%h:%%p


There's a lot of stuff there. 

There will be more ;) but for now let's stick with that.

## Inventories

In [15]:
# inventory - contains a static list of nodes to contact. 
# An inventory can be static or dynamic, as we will see in the following lessons

!cat inventory

#
# This inventory file contains a list of server to 
#  play with - divided in groups.
#
[course]
# this is the local machine where you run jupyter
# the tutorial just works with this one.
pythonforsysadmin_course_1  ansible_connection=local


# Another group of servers
# where we can pass optional arguments
# Homework: you can play with this group of host
#  once you exchange ssh-keys between the pythonforsysadmin_course_1
#  container and the pythonforsysadmin_ansible_* ones._
[ansible]
172.17.0.[5:7]   



In [17]:
# You can have many inventory files
!cat staging

#
# This inventory file contains a list of server to 
#  play with - divided in groups.
#
[course]
# this is the local machine where you run jupyter
# the tutorial just works with this one.
pythonforsysadmin_course_1  ansible_connection=local



## Environment variables

In [16]:
# group_vars - a directory containing environment files for various hosts
!tree group_vars

group_vars
├── all
└── staging

0 directories, 2 files


In [ ]:
# I set  env_name in two different files
!grep env_name -r group_vars/



In [28]:
# The debug module (-m debug) shows a variable.
#   by default uses the inventory set into ansible.cfg, thus dumping
!ansible all -m debug -a 'var=env_name'


group_vars/inventory:env_name: prod
group_vars/staging:env_name: staging
group_vars/all:env_name: default
pythonforsysadmin_course_1 | SUCCESS => {
    "env_name": "default"
}
172.17.0.6 | SUCCESS => {
    "env_name": "default"
}
172.17.0.5 | SUCCESS => {
    "env_name": "default"
}
172.17.0.7 | SUCCESS => {
    "env_name": "default"
}
pythonforsysadmin_course_1 | SUCCESS => {
    "env_name": "staging"
}


In [31]:
#
# Exercise: Dump env_name tied to the staging inventory.
#
# - which is the expected output?
# - what ties the "staging" inventory file to group_vars/staging?

!ansible all -i staging -m debug -a 'var=env_name'

pythonforsysadmin_course_1 | SUCCESS => {
    "env_name": "staging"
}


In [33]:
#
# Which is the expected output of
#
!ansible course -i staging -m debug -a 'var=proxy_env'

pythonforsysadmin_course_1 | SUCCESS => {
    "proxy_env": {
        "http_proxy": "http://proxy:3128", 
        "https_proxy": "http://proxy:3128", 
        "no_proxy": "127.0.0.1,localhost,.example.com"
    }
}
